In [1]:
# Solar Simulation Demo
# By Rohit Sharma, IIT Kanpur
#------------------------------
# Set up karabo environment: https://i4ds.github.io/Karabo-Pipeline/installation_user.html 


import numpy as np
from astropy.io import fits
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging import imager_wsclean
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta

**Telescope Configuration settings:** Path to tm files of the telescopes
More telescopes: "https://github.com/i4Ds/Karabo-Pipeline/tree/main/karabo/data"

In [2]:
#telescope_path = '/media/rohit/sdata/ska-solar-files/meerkat.tm'
telescope_path = '/data/rohit/skao_repo/ska1mid.tm'
#telescope_path = '/data/rohit/skao_repo/ska-mid-AAstar.tm'
#telescope_path = '/data/rohit/skao_repo/ska-mid-AA2.tm'
#telescope_path = '/data/rohit/skao_repo/ska-mid-AA1.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/mwa.phase1.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1low.tm'


**Skymodel Settings:**
+ Path to the fits to the models, incase of an image
+ Takes numpy array as inputs with minimum three columns of the source properties, i.e. RA (deg), DEC (deg), and Flux density (Jy)

In [2]:
path = '/data/rohit/ska-mid-sim-demo/'
start_frequency_hz_ = 240*1.e6
beam_size_arcsec = 1
aa=fits.open('/data/rohit/20151203_240MHz_psimas.fits')
ms_filename = path+"solar_240MHz_ska-mid.ms"
vis_filename = path+"solar_freq_240MHz_ska-mid.vis"
solar_map=aa[0].data;solar_map_jy=solar_map/np.nanmax(solar_map)*20*1.e4*(start_frequency_hz_/2.4e8)**1
ra_sun_center=249.141666667;dec_sun_center=21.986 #16 34 34.52 -21 59 09.7
ra_grid,dec_grid=np.meshgrid((np.arange(256)-128)*22.5/3600.,(np.arange(256)-128)*22.5/3600.)
ra_grid=ra_grid+ra_sun_center;dec_grid=dec_grid+dec_sun_center
idx=np.where(solar_map>0.1*np.nanmax(solar_map))
sky_model_ra=ra_grid[idx];sky_model_dec=dec_grid[idx];flux=solar_map_jy[idx]

**Run Simulation**

In [4]:

# Simulation Settings 
hour_=0
minutes_=1
noise_enable_ = False
enable_array_beam=False
ntchan =1
nchan = 1
sky = SkyModel()
sky_data = np.array([sky_model_ra, sky_model_dec, flux,np.zeros(len(flux)), \
    np.zeros(len(flux)),np.zeros(len(flux)), np.zeros(len(flux)),np.zeros(len(flux)), \
np.zeros(len(flux)),np.zeros(len(flux)), np.zeros(len(flux)),np.zeros(len(flux))]).T
sky_data=sky_data[0:16000,:]
sky.add_point_sources(sky_data)
#telescope_name="SKA1MID"
#telescope=Telescope.get_SKA1_LOW_Telescope()
dtime=datetime(2000, 1, 1, 10, 0, 00, 0) # MeerKAT/ SKA-mid
#dtime=datetime(2000, 1, 1, 4, 0, 00, 0) # MWA / SKA-low
backend=SimulatorBackend.OSKAR
#telescope = Telescope.constructor(telescope_name, backend=backend)
telescope=Telescope.read_OSKAR_tm_file(telescope_path)
telescope.read_OSKAR_tm_file(telescope_path)
simulation = InterferometerSimulation(vis_path=vis_filename, ms_file_path=ms_filename,
    channel_bandwidth_hz=1, time_average_sec=10, noise_enable=noise_enable_, use_gpus=True,
    noise_seed="time", noise_freq="Range", noise_rms="Range", 
    noise_start_freq=1.e9, noise_inc_freq=1.e8, noise_number_freq=24, 
    noise_rms_start=0, noise_rms_end=0, enable_numerical_beam=enable_array_beam,
    enable_array_beam=enable_array_beam) 
observation = Observation(mode='Tracking',phase_centre_ra_deg=ra_sun_center, start_date_and_time=dtime, 
    length=timedelta(hours=hour_, minutes=minutes_, seconds=0, milliseconds=0), 
    phase_centre_dec_deg=dec_sun_center, number_of_time_steps=1, 
    start_frequency_hz=start_frequency_hz_, frequency_increment_hz=1, 
    number_of_channels=1, ) 

# Run the interferometer
visibility = simulation.run_simulation(telescope, sky, observation, backend=backend)


W|                                                                   
W|== WARNING: No GPU capability available.
W|                                                                   
Saved visibility to /data/rohit/ska-mid-sim-demo/solar_freq_240MHz_ska-mid.vis


**Imaging from the Visibilities**

In [ ]:

# Imaging Settings & Using WSCLEAN

npix_per_beam = 3
imgsize=2096
cellsize_arcsec=2 #beam_size_arcsec/npix_per_beam
cellsize_rad=cellsize_arcsec/3600*np.pi/180 # in rad
dirty_imager = OskarDirtyImager(
    OskarDirtyImagerConfig(
        imaging_npixel=imgsize,
        imaging_cellsize=cellsize_rad,
    ))
dirty_oskar_img = path+"solar_freq_240MHz_ska-mid_oskar.fits"
dirty_image = dirty_imager.create_dirty_image(visibility,output_fits_path=dirty_oskar_img)
dirty_wsclean_img = path+"solar_freq_240MHz_ska-mid_wsclean.fits"
img_cmd = 'wsclean \
        -size '+str(imgsize)+' '+str(imgsize)+' \
        -name '+dirty_wsclean_img+' \
        -scale '+str(cellsize_rad)+'rad -niter 2 -mgain 0.8 \
        -weight uniform\
        -maxuv-l 9000 -minuv-l 10\
        -channels-out '+str(nchan)+' '+ms_filename
print(img_cmd)
try:
    restored = imager_wsclean.create_image_custom_command(command=img_cmd)
except:
    pass # doing nothing on exception       

W|                                                                   
W|== WARNING: Skipped 542 visibility points.
W|                                                                   


/home/rohit/miniconda3/envs/karabo/lib/python3.9/site-packages/karabo/imaging/image.py:119: UserWarning: Received 3D data for image object.
                Will assume the 3 axes correspond to
                (polarisations, pixels_x, pixels_y).
                Inserting 1 additional axis for frequencies.
  warnings.warn(


wsclean         -size 2096 2096         -name /data/rohit/ska-mid-sim-demo/solar_freq_240MHz_ska-mid_wsclean.fits         -scale 9.69627362219072e-06rad -niter 2 -mgain 0.8         -weight uniform        -maxuv-l 9000 -minuv-l 10        -channels-out 1 /data/rohit/ska-mid-sim-demo/solar_240MHz_ska-mid.ms
Creating /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-HiPXT8zvvZ for Disk cache for WSClean custom command images
WSClean command: [cd /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-HiPXT8zvvZ && OPENBLAS_NUM_THREADS=1 wsclean         -size 2096 2096         -name /data/rohit/ska-mid-sim-demo/solar_freq_240MHz_ska-mid_wsclean.fits         -scale 9.69627362219072e-06rad -niter 2 -mgain 0.8         -weight uniform        -maxuv-l 9000 -minuv-l 10        -channels-out 1 /data/rohit/ska-mid-sim-demo/solar_240MHz_ska-mid.ms]
WSClean output:
[
WSClean version 3.4 (2023-10-11)
This software package is released under the GPL version 3.
Author: André Offringa (offringa@gmail.com).

No corre